합성곱 관련!(https://wikidocs.net/63565)

In [27]:
import torch
import torch.nn as nn

In [28]:
# 배치 크기 x 채널 x 높이(height)x너비(width)의 크기의 텐서를 선언
# pytorch 에서는 체널, 높이, 너비 순으로 표현한고 openCV에서는 높이, 너비 ,채널 순으로 표현한다.
inputs = torch.Tensor(1,1,28,28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


합성곱층과 풀링 선언하기
- 첫번째 합성곱 층 구현(1채널 짜리를 입력받아서 32채널 뽑아내는 커널 사이즈는 3이고 패딩은 1)

In [29]:
conv1 = nn.Conv2d(1,32,3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


- 두번째 합성곱 추가(32채널 짜리를 입렵바아서 64채널을 뽑아내는데 커널사이지는 3이고 패딩은 1)

In [30]:
conv2 = nn.Conv2d(32,64,kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


- 맥스풀링을 구현(정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘다 해당 겂으로 지정)

In [31]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


구현채를 연결하여 모델만들기 : 지금까지는 선언만한 것이고 아직 이들을 연결시키지는 않았습니다. 이들은 연결시켜서 모델을 완성

In [32]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [33]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [34]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [35]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [36]:
out = out.view(out.size(0), -1) 
print(out.shape)

torch.Size([1, 3136])


In [37]:
fc = nn.Linear(3136, 10) # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [38]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [40]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [42]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [43]:
# 데이터로더를 사용하여 배치 크기를 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

이제 클래스로 모델 설계

In [44]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

CNN 모델 정의

In [45]:
model = CNN().to(device)